<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/Modelo_3_Red_Neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generación de texto**

# **Red Neuronal Recurrente**

En este avance se buscar entrenar una red neuronal recurrente para generar texto, carácter a carácter.

In [1]:
!pip install chardet

In [2]:
debug = False
debug2 = False

In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import random
import chardet
import warnings
warnings.simplefilter('ignore')
scaler = torch.cuda.amp.GradScaler()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
def random_seed(SEED):

    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

SEED = 508
random_seed(SEED)

In [5]:
from google.colab import drive
import pandas as pd
from google.colab import drive
import shutil
from google.colab import drive
#import nltk

#Aquí seleccionamos el archivo desde nuestro equipo.

from google.colab import files
uploaded = files.upload()

Saving df_total.csv to df_total.csv


In [6]:
data0=pd.read_csv('df_total.csv')

In [ ]:
df = data0.loc[data0["Type"]=="Macroeconomia"]

In [ ]:
df = df.news[0:50]

In [ ]:
df = df.to_csv("data_export.txt", sep="\t", index=False)

In [ ]:
f = open("data_export.txt", "r", encoding='utf-8')
text = f.read()
text[:300], len(text)

('news\nCon el dato de marzo el IPC interanual encadena su decimoquinta tasa positiva consecutiva. La inflación publicada por el INE se ha mantenido igual respecto al avance del 30 de marzo y se sitúa 22 puntos por encima del dato de febrero que ascendió al 76.De esos 22 puntos de diferencia la mayor p',
 157300)

Tokenización

In [ ]:
import string

all_characters = string.printable + "ñÑáÁéÉíÍóÓúÚ¿¡"
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0cñÑáÁéÉíÍóÓúÚ¿¡'

In [ ]:
import string

class Tokenizer():

  def __init__(self):
    self.all_characters = all_characters
    self.n_characters = len(self.all_characters)

  def text_to_seq(self, string):
    seq = []
    for c in range(len(string)):
        try:
            seq.append(self.all_characters.index(string[c]))
        except:
            continue
    return seq

  def seq_to_text(self, seq):
    text = ''
    for c in range(len(seq)):
        text += self.all_characters[seq[c]]
    return text

tokenizer = Tokenizer()
tokenizer.n_characters

114

Ahora podemos tokenizar todo el texto.

In [ ]:
text_encoded = tokenizer.text_to_seq(text)

El Dataset

In [ ]:
train_size = len(text_encoded) * 80 // 100
train = text_encoded[:train_size]
test = text_encoded[train_size:]

len(train), len(test)

(125839, 31460)

Para entrenar nuestra red, vamos a necesitar secuencias de texto de una longitud determinada. Podemos generar estas ventanas con la siguiente función

In [ ]:
import random

def windows(text, window_size = 100):
    start_index = 0
    end_index = len(text) - window_size
    text_windows = []
    while start_index < end_index:
      text_windows.append(text[start_index:start_index+window_size+1])
      start_index += 1
    return text_windows

text_encoded_windows = windows(text_encoded)

Ejemplo de las ventanas con las que trabajará el modelo

In [ ]:
print(tokenizer.seq_to_text((text_encoded_windows[0])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[1])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[2])))

news
Con el dato de marzo el IPC interanual encadena su decimoquinta tasa positiva consecutiva. La in

ews
Con el dato de marzo el IPC interanual encadena su decimoquinta tasa positiva consecutiva. La inf

ws
Con el dato de marzo el IPC interanual encadena su decimoquinta tasa positiva consecutiva. La infl


In [ ]:
import torch

class CharRNNDataset(torch.utils.data.Dataset):
  def __init__(self, text_encoded_windows, train=True):
    self.text = text_encoded_windows
    self.train = train

  def __len__(self):
    return len(self.text)

  def __getitem__(self, ix):
    if self.train:
      return torch.tensor(self.text[ix][:-1]), torch.tensor(self.text[ix][-1])
    return torch.tensor(self.text[ix])

In [ ]:
train_text_encoded_windows = windows(train)
test_text_encoded_windows = windows(test)

dataset = {
    'train': CharRNNDataset(train_text_encoded_windows),
    'val': CharRNNDataset(test_text_encoded_windows)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=512, shuffle=True, pin_memory=True),
    'val': torch.utils.data.DataLoader(dataset['val'], batch_size=2048, shuffle=False, pin_memory=True),
}

len(dataset['train']), len(dataset['val'])

(125739, 31360)

In [ ]:
input, output = dataset['train'][0]
tokenizer.seq_to_text(input)

'news\nCon el dato de marzo el IPC interanual encadena su decimoquinta tasa positiva consecutiva. La i'

In [ ]:
tokenizer.seq_to_text([output])

'n'

Embeddings

Si bien hemos conseguido convertir nuestro texto a números, una red neuronal seguirá sin ser capaz de trabajar con nuestros datos ya que, como hemos visto en posts anteriores, éstos tienen que estar normalizados. Además, en función del tokenizador que utilicemos es posible que el mismo carácter tenga asociados diferentes valores. Es por esto que necesitamos codificar nuestro texto de alguna manera.

Una opción puede ser el one-hot encoding, al fin y al cabo podemos considerar cada letra como una categoría y que nuestra red nos de a la salida una distribución de probabilidad sobre todos los posibles carácteres.

In [ ]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=128, hidden_size=256, num_layers=2, dropout=0.2):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)
    y = self.fc(x[:,-1,:])
    return y

Nuestro modelo recibirá batches de frases con el índice de cada palabra que nos proporciona el tokenizador. A la salida tendremos una distribución de probabilidad sobre todos los posibles carácteres para cada frase del batch.

In [ ]:
model = CharRNN(input_size=tokenizer.n_characters)
outputs = model(torch.randint(0, tokenizer.n_characters, (64, 50)))
outputs.shape

torch.Size([64, 114])

Entrenamiento

In [ ]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader['val'])
        val_loss = []
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval()
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

In [ ]:
model = CharRNN(input_size=tokenizer.n_characters)
fit(model, dataloader, epochs=10)

val_loss 2.11099: 100%|██████████| 16/16 [02:53<00:00, 10.86s/it]


Epoch 1/10 loss 2.55778 val_loss 2.11099


val_loss 1.88356: 100%|██████████| 16/16 [02:52<00:00, 10.81s/it]


Epoch 2/10 loss 1.99905 val_loss 1.88356


val_loss 1.75608: 100%|██████████| 16/16 [02:53<00:00, 10.83s/it]


...

Epoch 8/10 loss 1.37424 val_loss 1.46050


val_loss 1.43130: 100%|██████████| 16/16 [02:54<00:00, 10.92s/it]


Epoch 9/10 loss 1.32539 val_loss 1.43130


val_loss 1.41380: 100%|██████████| 16/16 [02:55<00:00, 11.00s/it]

Epoch 10/10 loss 1.28499 val_loss 1.41380


Genrando texto

In [ ]:
X_new = "El desempleo, "
X_new_encoded = tokenizer.text_to_seq(X_new)
y_pred = predict(model, X_new_encoded)
y_pred = torch.argmax(y_pred, axis=1)[0].item()
tokenizer.seq_to_text([y_pred])

'a'

In [ ]:
for i in range(100):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = torch.argmax(y_pred, axis=1)[0].item()
  X_new += tokenizer.seq_to_text([y_pred])

X_new

'El desempleo, a la inflación de la inflación de la inflación de la inflación de la inflación de la inflación de la'

In [ ]:
temp=1
for i in range(10000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

El desempleo, a la inflación de la inflación de la inflación de la inflación de la inflación de la inflación de la famíicla Cambilo operables ha haya el producido en los del aún los básicos en tormde Anterior desturales.Sin destularon los bajos anteriores en los merestógenque la empleos que ha de 26 y cuaurrollopera una también pubijo en Comunidad en explico aún seítico una todo los sectoras del contliciones entre esciente de Alterioa a menorsucitaciones. En mayor sectuvará.Alistas. El impacto la ingresarias menor santadas recusos fectores a silimendo y baja cuásticos como el países lechó en la Reserva novilará año del Estos laros necesarios que hago mantenera que se Comunisión.Los precios sobre una inflacia actual más francia des0guiliaral rechor productores hive.Si una está para informanidezadó que restación con los nuevos ingarte de el precio un comportamión vamos más que Mintería que de incertes para elloz bave la acorta y el CCg estre el sector guVmo. Animilutorio Canshario autoun

In [ ]:
temp=0.7
for i in range(10000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

El desempleo, a la inflación de la inflación de la inflación de la inflación de la inflación de la inflación de la famíicla Cambilo operables ha haya el producido en los del aún los básicos en tormde Anterior desturales.Sin destularon los bajos anteriores en los merestógenque la empleos que ha de 26 y cuaurrollopera una también pubijo en Comunidad en explico aún seítico una todo los sectoras del contliciones entre esciente de Alterioa a menorsucitaciones. En mayor sectuvará.Alistas. El impacto la ingresarias menor santadas recusos fectores a silimendo y baja cuásticos como el países lechó en la Reserva novilará año del Estos laros necesarios que hago mantenera que se Comunisión.Los precios sobre una inflacia actual más francia des0guiliaral rechor productores hive.Si una está para informanidezadó que restación con los nuevos ingarte de el precio un comportamión vamos más que Mintería que de incertes para elloz bave la acorta y el CCg estre el sector guVmo. Animilutorio Canshario autoun

In [ ]:
temp=0.5
for i in range(10000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

El desempleo, a la inflación de la inflación de la inflación de la inflación de la inflación de la inflación de la famíicla Cambilo operables ha haya el producido en los del aún los básicos en tormde Anterior desturales.Sin destularon los bajos anteriores en los merestógenque la empleos que ha de 26 y cuaurrollopera una también pubijo en Comunidad en explico aún seítico una todo los sectoras del contliciones entre esciente de Alterioa a menorsucitaciones. En mayor sectuvará.Alistas. El impacto la ingresarias menor santadas recusos fectores a silimendo y baja cuásticos como el países lechó en la Reserva novilará año del Estos laros necesarios que hago mantenera que se Comunisión.Los precios sobre una inflacia actual más francia des0guiliaral rechor productores hive.Si una está para informanidezadó que restación con los nuevos ingarte de el precio un comportamión vamos más que Mintería que de incertes para elloz bave la acorta y el CCg estre el sector guVmo. Animilutorio Canshario autoun

In [ ]:
temp=0.3
for i in range(10000):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

El desempleo, a la inflación de la inflación de la inflación de la inflación de la inflación de la inflación de la famíicla Cambilo operables ha haya el producido en los del aún los básicos en tormde Anterior desturales.Sin destularon los bajos anteriores en los merestógenque la empleos que ha de 26 y cuaurrollopera una también pubijo en Comunidad en explico aún seítico una todo los sectoras del contliciones entre esciente de Alterioa a menorsucitaciones. En mayor sectuvará.Alistas. El impacto la ingresarias menor santadas recusos fectores a silimendo y baja cuásticos como el países lechó en la Reserva novilará año del Estos laros necesarios que hago mantenera que se Comunisión.Los precios sobre una inflacia actual más francia des0guiliaral rechor productores hive.Si una está para informanidezadó que restación con los nuevos ingarte de el precio un comportamión vamos más que Mintería que de incertes para elloz bave la acorta y el CCg estre el sector guVmo. Animilutorio Canshario autoun